In [3]:
# Supress Warnings 
import warnings
warnings.filterwarnings('ignore')
# Plotting
import matplotlib.pyplot as plt
# Data science
import pandas as pd
import numpy as np
# Geospatial
import contextily as cx
import xarray as xr
import zarr # Not referenced, but required for xarray
# Import Planetary Computer tools
import fsspec
import pystac
# Other
import os
import zipfile
from itertools import cycle

In [94]:
# read in the stuff we need to join. Read in as dataarray, not as dataset, so that we can use .rename('data')
weather_data1 = xr.open_dataarray('weather_data_1012.nc')
weather_data2 = xr.open_dataarray('weather_data_1314.nc')
weather_data3 = xr.open_dataarray('weather_data_1516.nc')
weather_data4 = xr.open_dataarray('weather_data_17.nc')
weather_data5 = xr.open_dataarray('weather_data_18.nc')
weather_data6 = xr.open_dataarray('weather_data_1920.nc')
frogs = pd.read_csv('30kfrogs.csv')
# drop 2021 and 2022 data
frogs = frogs[frogs['eventDate'] < '2021-01-01 00:00:00']

**1. Try to drop null values along band(fail)**

**2. Try to merge all 6 datasets/ dataarrays(also fail)**

**3. Try to run join function before merging (work)**

In [95]:
def join_frogs(frogs, data):
    """Collects the data for each frog location and joins it onto the frog data 
    Arguments:
    frogs -- dataframe containing the response variable along with ["decimalLongitude", "decimalLatitude", "key"]
    data -- xarray dataarray of features, indexed with geocoordinates
    """
    return frogs.merge(
        (data.rename('data').sel(
                x=xr.DataArray(frogs.decimalLongitude, dims="key", coords={"key": frogs.key}), 
                y=xr.DataArray(frogs.decimalLatitude, dims="key", coords={"key": frogs.key}),
                method="nearest"
            )
            .to_dataframe()
            .assign(val = lambda x: x.iloc[:, -1])
            [['val']]
            .reset_index()
            .drop_duplicates()
            .pivot(index="key", columns="band", values="val")
            .reset_index()
        ),
        on = ['key'],
        how = 'inner'
    )

In [96]:
climatefrog1012 = join_frogs(frogs[frogs['eventDate'] < '2013-01-01 00:00:00']
                             , weather_data1)
climatefrog1012.to_csv('climate_frog1012.csv')

climatefrog1314 = join_frogs(frogs[('2012-12-31 23:59:59'< frogs['eventDate']) & ( frogs['eventDate']< '2015-01-01 00:00:00')]
                             , weather_data2)
climatefrog1314.to_csv('climate_frog1314.csv')

climatefrog1516 = join_frogs(frogs[('2014-12-31 23:59:59'< frogs['eventDate']) & ( frogs['eventDate']< '2017-01-01 00:00:00')]
                            , weather_data3)
climatefrog1516.to_csv('climate_frog1516.csv')

climatefrog17 = join_frogs(frogs[('2016-12-31 23:59:59'< frogs['eventDate']) & ( frogs['eventDate']< '2018-01-01 00:00:00')]
                           , weather_data4)
climatefrog17.to_csv('climate_frog17.csv')

climatefrog18 = join_frogs(frogs[('2017-12-31 23:59:59'< frogs['eventDate']) & ( frogs['eventDate']< '2019-01-01 00:00:00')]
                           , weather_data5)
climatefrog18.to_csv('climate_frog18.csv')

climatefrog1920 = join_frogs(frogs[('2018-12-31 23:59:59'< frogs['eventDate']) & ( frogs['eventDate']< '2021-01-01 00:00:00')]
                             , weather_data6)
climatefrog1920.to_csv('climate_frog1920.csv')

In [97]:
# Now we can join the 6 dfs using pandas. First check their shape
for i in [climatefrog1012,climatefrog1314,climatefrog1516,climatefrog17,climatefrog18,climatefrog1920]:
    print(i.shape)
climatefrog1020 = pd.concat([climatefrog1012,climatefrog1314,climatefrog1516,climatefrog17,climatefrog18,climatefrog1920])
climatefrog1020.to_csv('climate_frog1020.csv')

(7879, 24)
(1896, 24)
(2359, 24)
(3615, 24)
(12527, 24)
(35066, 24)
